In [ ]:
import os, sys
from sklearn.model_selection import KFold
from tqdm import tqdm
import json
from google.colab import drive
import numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split

import itertools
import random


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Mon Apr 26 10:57:06 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
drive.mount('/content/drive')
nb_path = '/content/notebooks' 
#os.symlink('/content/drive/My Drive/Colab Notebooks', nb_path)
sys.path.insert(0,nb_path)

Mounted at /content/drive


In [ ]:
!pip install fire
import fire

     |████████████████████████████████| 92kB 5.8MB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115928 sha256=12f11ce0719176f5d09a60f327c0f04de6070f6c28d96632012f621bb5870949
  Stored in directory: /root/.cache/pip/wheels/af/19/30/1ea0cad502dcb4e66ed5a690279628c827aea38bbbab75d5ed
Successfully built fire


In [ ]:
!pip install --target=$nb_path jdc

In [ ]:
!pip install -r /content/drive/My\ Drive/Colab\ Notebooks/SpanBERT/requirements.txt

     |████████████████████████████████| 51kB 4.1MB/s 
     |████████████████████████████████| 133kB 10.1MB/s 
     |████████████████████████████████| 5.7MB 10.7MB/s 
     |████████████████████████████████| 163kB 48.3MB/s 
     |████████████████████████████████| 552kB 59.0MB/s 
     |████████████████████████████████| 61kB 7.7MB/s 
     |████████████████████████████████| 788kB 46.5MB/s 
     |████████████████████████████████| 1.0MB 50.0MB/s 
     |████████████████████████████████| 20.3MB 157kB/s 
     |████████████████████████████████| 102kB 13.1MB/s 
     |████████████████████████████████| 61kB 9.6MB/s 
     |████████████████████████████████| 2.1MB 54.4MB/s 
     |████████████████████████████████| 348kB 53.4MB/s 
     |████████████████████████████████| 901kB 48.6MB/s 
     |████████████████████████████████| 235kB 59.8MB/s 
     |████████████████████████████████| 133kB 62.6MB/s 
     |████████████████████████████████| 675kB 48.9MB/s 
     |████████████████████████████████| 61kB 8.8MB/s 


In [ ]:
!pip install boto3

## Загрузим данные

In [ ]:
source ="/content/drive/MyDrive/Datasets/NREL/all_data_task1.json"
#dest_dir = "/content/drive/MyDrive/Datasets/wiki_news_25_12/fold"

In [ ]:
all_data = json.load(open(source))

data = [ex for ex in all_data if len(ex['token']) < 150]
start_data = data

In [ ]:
len(data)

13621

In [ ]:
def print_relations_stats(examples):
    relations = Counter([x['relation'] for x in examples])
    print(relations.most_common())


examples = []
n_pos_examples, n_neg_examples = 0, 0
positions = [0, 1]
# subj_ners = ['GROUP', 'PERSON', 'ORGANIZATION', 'CITY', 'COUNTRY', 'GPE', 'PROFESSION', 'REGION']
#subj_ners = ['PROFESSION', 'PERSON']
data = [example for example in data if
        all([subj_key in relation for subj_key in ['subj_type', 'obj_type'] for relation in example['relations']])]
subj_ners = set([rel['subj_type'] for relations in data for rel in relations['relations']])
for example in data:
    pair_starts = set()
    for relation in example['relations']:
        new_example = {}
        try:
            new_example['token'] = example['token']
            new_example['relation'] = relation['relation']
            new_example['subj_start'] = relation['subj_start']
            new_example['subj_end'] = relation['subj_end']
            new_example['obj_start'] = relation['obj_start']
            new_example['obj_end'] = relation['obj_end']
            new_example['subj_type'] = relation['subj_type']
            new_example['obj_type'] = relation['obj_type']
            new_example['stanford_ner'] = example['stanford_ner']
            filename = example['filename'].split('/')[-1][:-4]
            new_example['filename'] = filename
            new_example['id'] = '{}_{}_{}'.format(filename, relation['subj_brat_code'], relation['obj_brat_code'])
        except KeyError:
            continue
        examples.append(new_example)
        n_pos_examples += 1
        pair_starts.add(tuple(sorted([relation['subj_start'], relation['obj_start']])))

    entities = [i for i, x in enumerate(example['stanford_ner']) if x.startswith('B-')]
    for pair in itertools.product(entities, repeat=2):
        pair = tuple(sorted(pair))
        if pair[0] == pair[1] or pair in pair_starts:
            continue
        if all([example['stanford_ner'][pair[i]][2:] not in subj_ners for i in positions]):
            continue
        pair_starts.add(pair)
        random.shuffle(positions)
        if example['stanford_ner'][pair[positions[0]]][2:] not in subj_ners:
            positions = positions[::-1]

        new_example = {}
        subj_end = pair[positions[0]] + 1
        while subj_end < len(example['stanford_ner']) and example['stanford_ner'][subj_end].startswith('I-'):
            subj_end += 1
        obj_end = pair[positions[1]] + 1
        while obj_end < len(example['stanford_ner']) and example['stanford_ner'][obj_end].startswith('I-'):
            obj_end += 1
        new_example['token'] = example['token']
        new_example['relation'] = 'no_relation'
        new_example['subj_start'] = pair[positions[0]]
        new_example['subj_end'] = subj_end - 1
        new_example['obj_start'] = pair[positions[1]]
        new_example['obj_end'] = obj_end - 1
        new_example['subj_type'] = example['stanford_ner'][pair[positions[0]]][2:]
        new_example['obj_type'] = example['stanford_ner'][pair[positions[1]]][2:]
        new_example['stanford_ner'] = example['stanford_ner']
        filename = example['filename'].split('/')[-1][:-4]
        new_example['filename'] = filename
        new_example['id'] = '{}_{}_{}'.format(filename, new_example['subj_start'], new_example['obj_start'])
        examples.append(new_example)
        n_neg_examples += 1

print(f'number of positive examples(предложений): {n_pos_examples}')
print(f'number of negative examples(предложений): {n_neg_examples}')
print(f'percentage of negative examples: {n_neg_examples * 100.0 / (n_pos_examples + n_neg_examples)}%')


number of positive examples(предложений): 20585
number of negative examples(предложений): 68972
percentage of negative examples: 77.01463872170797%


In [ ]:
len(data)

13583

In [ ]:
len(examples)

89557

In [ ]:
examples[1]

{'filename': '40819_text',
 'id': '40819_text_0_10',
 'obj_end': 11,
 'obj_start': 10,
 'obj_type': 'PERSON',
 'relation': 'no_relation',
 'stanford_ner': ['B-PROFESSION',
  'I-PROFESSION',
  'B-PERSON',
  'I-PERSON',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-PERSON',
  'I-PERSON',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 'subj_end': 1,
 'subj_start': 0,
 'subj_type': 'PROFESSION',
 'token': ['Южноафриканский',
  'президент',
  'Джейкоб',
  'Зума',
  'заявил',
  ',',
  'что',
  'сообщения',
  'о',
  'выписке',
  'Нельсона',
  'Манделы',
  'из',
  'больницы',
  'не',
  'соответствуют',
  'действительности',
  '.']}

In [ ]:
full_data = [ex for ex in all_data if len(ex['token']) < 100]
problem_data = set(full_data) - set(data)

TypeError: ignored

In [ ]:
kf = KFold(n_splits=4, shuffle=True, random_state=42)
i = 0
total_data = []
for train_index, test_index in tqdm(kf.split(range(len(examples)))):
    test_examples_index, dev_examples_index = train_test_split(test_index, test_size=0.5, shuffle=True, random_state=42)

    examples = np.array(examples)
    print('fold {}'.format(i))
    train_examples = list(examples[train_index])
    test_examples = list(examples[test_examples_index])
    dev_examples = list(examples[dev_examples_index])

    total_data.append((train_examples, dev_examples, test_examples))

    print_relations_stats(train_examples)
    print_relations_stats(dev_examples)
    print_relations_stats(test_examples)
    '''
    json.dump(train_examples, open(os.path.join(dest_dir + str(i), 'train.json'), 'w'))
    json.dump(test_examples, open(os.path.join(dest_dir + str(i), 'test.json'), 'w'))
    json.dump(dev_examples, open(os.path.join(dest_dir + str(i), 'dev.json'), 'w'))
    '''

    i += 1

4it [00:00, 19.97it/s]

fold 0
[('no_relation', 51763), ('WORKS_AS', 2549), ('PARTICIPANT_IN', 2277), ('POINT_IN_TIME', 1223), ('TAKES_PLACE_IN', 1045), ('AGENT', 899), ('ORIGINS_FROM', 846), ('WORKPLACE', 613), ('AGE_IS', 610), ('LOCATED_IN', 572), ('ALTERNATIVE_NAME', 494), ('HAS_CAUSE', 391), ('AWARDED_WITH', 299), ('PRODUCES', 294), ('CONVICTED_OF', 220), ('KNOWS', 207), ('HEADQUARTERED_IN', 202), ('MEDICAL_CONDITION', 179), ('INANIMATE_INVOLVED', 176), ('PLACE_RESIDES_IN', 158), ('PARENT_OF', 156), ('SUBEVENT_OF', 153), ('MEMBER_OF', 145), ('IDEOLOGY_OF', 123), ('ORGANIZES', 114), ('PENALIZED_AS', 110), ('DATE_OF_BIRTH', 109), ('SPOUSE', 102), ('DATE_OF_CREATION', 97), ('PLACE_OF_BIRTH', 90), ('DATE_OF_DEATH', 86), ('AGE_DIED_AT', 85), ('SUBORDINATE_OF', 80), ('SIBLING', 75), ('OWNER_OF', 72), ('FOUNDED_BY', 71), ('ABBREVIATION', 70), ('PLACE_OF_DEATH', 65), ('DATE_FOUNDED_IN', 43), ('CAUSE_OF_DEATH', 43), ('SCHOOLS_ATTENDED', 40), ('START_TIME', 36), ('PART_OF', 33), ('RELATIVE', 29), ('EXPENDITURE', 25

In [ ]:
# срез датасета по официальному разбиению 
import pandas as pd
docs_info = pd.read_pickle('/content/drive/MyDrive/Datasets/NREL/docs_info_final.pkl')

In [ ]:
docs_info1 = docs_info
docs_info1['file_name'] = docs_info.index

In [ ]:
docs_split = docs_info1.groupby('fold')['file_name'].apply(list)
docs_split

fold
dev      [218392_text, 598378_text, 211066_text, 349503...
test     [1160, 2021, 546860_text, 184456_text, 209731_...
train    [40819_text, 13162_text, 106656_text, 520080_t...
Name: file_name, dtype: object

In [ ]:
print(set(docs_split['dev']).intersection(docs_split['train']))
print(set(docs_split['test']).intersection(docs_split['train']))
print(set(docs_split['dev']).intersection(docs_split['test']))
# множества не пересекаются что хорошо

train_start_len = len(docs_split['train'])
test_start_len = len(docs_split['test'])
dev_start_len = len(docs_split['dev'])
total_num = train_start_len + test_start_len + dev_start_len

print('В начальной выборке: ')
print('train:',train_start_len, '~', train_start_len / total_num)
print('dev:', dev_start_len, '~', dev_start_len / total_num)
print('test:', test_start_len, '~',test_start_len / total_num)

{'120043_text'}
{'141989_text', '140608_text'}
set()
В начальной выборке: 
train: 749 ~ 0.8002136752136753
dev: 94 ~ 0.10042735042735043
test: 93 ~ 0.09935897435897435


In [ ]:
# удалим пересечения

docs_info = docs_info[~docs_info['file_name'].duplicated(keep='first')]

In [ ]:
docs_folds = docs_info['fold']
train_official_ex = []
test_official_ex = []
dev_official_ex = []
                 

for i, ex in enumerate(examples):
  fold = docs_folds.loc[ex['filename']]
  try:
    if fold == 'train':
      train_official_ex.append(ex)
    elif fold == 'test':
      test_official_ex.append(ex)
    elif fold == 'dev':
      dev_official_ex.append(ex)
    else:
      print("error")
  except:
    print(fold)
random.shuffle(train_official_ex)
random.shuffle(test_official_ex)
random.shuffle(dev_official_ex)

In [ ]:
train_len = len(train_official_ex)
test_len = len(test_official_ex)
dev_len = len(dev_official_ex)
total_len = train_len + dev_len + test_len

print('На вход нейронной сети(после препроцессинга): ')
print('train: {} ~ {}'.format(train_len, train_len / total_len))
print('dev: {} ~ {}'.format(dev_len, dev_len / total_len))
print('test: {} ~ {}'.format(test_len, test_len / total_len))

На вход нейронной сети(после препроцессинга): 
train: 71257 ~ 0.795660864030729
dev: 8983 ~ 0.10030483379300334
test: 9317 ~ 0.10403430217626763


In [ ]:
total_data= []
total_data.append((train_official_ex, dev_official_ex, test_official_ex))

In [ ]:
len(total_data)

1

In [ ]:
data_folder = '/content/drive/MyDrive/Datasets/NREL/task1'
with open(os.path.join(data_folder, 'train.json'), 'r')  as f:
  json.dump(train_official_ex, f)
with open(os.path.join(data_folder, 'dev.json'), 'r')  as f:
  json.dump(dev_official_ex, f)
with open(os.path.join(data_folder, 'test.json'), 'r')  as f:
  json.dump(test_official_ex, f)

NameError: ignored

In [ ]:
data_folder = '/content/drive/MyDrive/Datasets/NREL/task1/data/json'
with open(os.path.join(data_folder, 'train.json'), 'r')  as f:
  train = json.load(f)
with open(os.path.join(data_folder, 'dev.json'), 'r')  as f:
  dev = json.load(f)
with open(os.path.join(data_folder, 'test.json'), 'r')  as f:
  test = json.load(f)

In [ ]:
total_data = [(train, dev, test)]

## Модель

In [ ]:
import sys


import argparse
import logging
import os
import random
import time
import json

import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset
from collections import Counter

import sys


from torch.nn import CrossEntropyLoss

from pytorch_pretrained_bert.file_utils import PYTORCH_PRETRAINED_BERT_CACHE, WEIGHTS_NAME, CONFIG_NAME
from pytorch_pretrained_bert.modeling import BertForSequenceClassification
from pytorch_pretrained_bert.tokenization import BertTokenizer
from pytorch_pretrained_bert.optimization import BertAdam,  WarmupLinearSchedule


In [ ]:
CLS = "[CLS]"
SEP = "[SEP]"

logging.basicConfig(format='%(asctime)s - %(levelname)s - %(name)s - %(message)s',
                    datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)
logger = logging.getLogger(__name__)


class InputExample(object):
    """A single training/test example for span pair classification."""

    def __init__(self, guid, sentence, span1, span2, ner1, ner2, label):
        self.guid = guid
        self.sentence = sentence
        self.span1 = span1
        self.span2 = span2
        self.ner1 = ner1
        self.ner2 = ner2
        self.label = label


class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, input_mask, segment_ids, label_id):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_id = label_id


class DataProcessor(object):
    """Processor for the TACRED data set."""

    @classmethod
    def _read_json(cls, input_file, fold):
        #with open(input_file, "r", encoding='utf-8') as reader:
        #    data = json.load(reader)
        if input_file == 'train':
          return total_data[fold][0]
        elif input_file == 'dev':
          return total_data[fold][1]
        elif input_file == 'test':
          return total_data[fold][2]
        else:
          print("!read_json")

    def get_train_examples(self, fold):
        """See base class."""
        return self._create_examples(
            self._read_json('train', fold), "train")

    def get_dev_examples(self, fold):
        """See base class."""
        return self._create_examples(
            self._read_json("dev", fold), "dev")

    def get_test_examples(self, fold):
        """See base class."""
        return self._create_examples(
            self._read_json( "test", fold), "test")

    def get_labels(self, fold, negative_label="no_relation"):
        """See base class."""
        dataset = self._read_json("train", fold)
        count = Counter()
        for example in dataset:
            count[example['relation']] += 1
        logger.info("%d labels" % len(count))
        # Make sure the negative label is alwyas 0
        labels = [negative_label]
        for label, count in count.most_common():
            logger.info("%s: %.2f%%" % (label, count * 100.0 / len(dataset)))
            if label not in labels:
                labels.append(label)
        return labels

    def _create_examples(self, dataset, set_type):
        """Creates examples for the training and dev sets."""
        examples = []
        for example in dataset:
            sentence = [convert_token(token) for token in example['token']]
            assert example['subj_start'] >= 0 and example['subj_start'] <= example['subj_end'] \
                and example['subj_end'] < len(sentence)
            assert example['obj_start'] >= 0 and example['obj_start'] <= example['obj_end'] \
                and example['obj_end'] < len(sentence)
            examples.append(InputExample(guid=example['id'],
                             sentence=sentence,
                             span1=(example['subj_start'], example['subj_end']),
                             span2=(example['obj_start'], example['obj_end']),
                             ner1=example['subj_type'],
                             ner2=example['obj_type'],
                             label=example['relation']))
        return examples


def convert_examples_to_features(examples, label2id, max_seq_length, tokenizer, special_tokens, mode='text'):
    """Loads a data file into a list of `InputBatch`s."""


    def get_special_token(w):
        if w not in special_tokens:
            special_tokens[w] = "[unused%d]" % (len(special_tokens) + 1)
        return special_tokens[w]

    num_tokens = 0
    num_fit_examples = 0
    num_shown_examples = 0
    features = []
    for (ex_index, example) in enumerate(examples):
        if ex_index % 10000 == 0:
            logger.info("Writing example %d of %d" % (ex_index, len(examples)))

        tokens = [CLS]
        SUBJECT_START = get_special_token("SUBJ_START")
        SUBJECT_END = get_special_token("SUBJ_END")
        OBJECT_START = get_special_token("OBJ_START")
        OBJECT_END = get_special_token("OBJ_END")
        SUBJECT_NER = get_special_token("SUBJ=%s" % example.ner1)
        OBJECT_NER = get_special_token("OBJ=%s" % example.ner2)

        if mode.startswith("text"):
            for i, token in enumerate(example.sentence):
                if i == example.span1[0]:
                    tokens.append(SUBJECT_START)
                if i == example.span2[0]:
                    tokens.append(OBJECT_START)
                for sub_token in tokenizer.tokenize(token):
                    tokens.append(sub_token)
                if i == example.span1[1]:
                    tokens.append(SUBJECT_END)
                if i == example.span2[1]:
                    tokens.append(OBJECT_END)
            if mode == "text_ner":
                tokens = tokens + [SEP, SUBJECT_NER, SEP, OBJECT_NER, SEP]
            else:
                tokens.append(SEP)
        else:
            subj_tokens = []
            obj_tokens = []
            for i, token in enumerate(example.sentence):
                if i == example.span1[0]:
                    tokens.append(SUBJECT_NER)
                if i == example.span2[0]:
                    tokens.append(OBJECT_NER)
                if (i >= example.span1[0]) and (i <= example.span1[1]):
                    for sub_token in tokenizer.tokenize(token):
                        subj_tokens.append(sub_token)
                elif (i >= example.span2[0]) and (i <= example.span2[1]):
                    for sub_token in tokenizer.tokenize(token):
                        obj_tokens.append(sub_token)
                else:
                    for sub_token in tokenizer.tokenize(token):
                        tokens.append(sub_token)
            if mode == "ner_text":
                tokens.append(SEP)
                for sub_token in subj_tokens:
                    tokens.append(sub_token)
                tokens.append(SEP)
                for sub_token in obj_tokens:
                    tokens.append(sub_token)
            tokens.append(SEP)
        num_tokens += len(tokens)

        if len(tokens) > max_seq_length:
            tokens = tokens[:max_seq_length]
        else:
            num_fit_examples += 1

        segment_ids = [0] * len(tokens)
        input_ids = tokenizer.convert_tokens_to_ids(tokens)
        input_mask = [1] * len(input_ids)
        padding = [0] * (max_seq_length - len(input_ids))
        input_ids += padding
        input_mask += padding
        segment_ids += padding
        label_id = label2id[example.label]
        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length

        if num_shown_examples < 20:
            if (ex_index < 5) or (label_id > 0):
                num_shown_examples += 1
                logger.info("*** Example ***")
                logger.info("guid: %s" % (example.guid))
                logger.info("tokens: %s" % " ".join(
                        [str(x) for x in tokens]))
                logger.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
                logger.info("input_mask: %s" % " ".join([str(x) for x in input_mask]))
                logger.info("segment_ids: %s" % " ".join([str(x) for x in segment_ids]))
                logger.info("label: %s (id = %d)" % (example.label, label_id))

        features.append(
                InputFeatures(input_ids=input_ids,
                              input_mask=input_mask,
                              segment_ids=segment_ids,
                              label_id=label_id))
    logger.info("Average #tokens: %.2f" % (num_tokens * 1.0 / len(examples)))
    logger.info("%d (%.2f %%) examples can fit max_seq_length = %d" % (num_fit_examples,
                num_fit_examples * 100.0 / len(examples), max_seq_length))
    return features


def convert_token(token):
    """ Convert PTB tokens to normal tokens """
    if (token.lower() == '-lrb-'):
            return '('
    elif (token.lower() == '-rrb-'):
        return ')'
    elif (token.lower() == '-lsb-'):
        return '['
    elif (token.lower() == '-rsb-'):
        return ']'
    elif (token.lower() == '-lcb-'):
        return '{'
    elif (token.lower() == '-rcb-'):
        return '}'
    return token


def simple_accuracy(preds, labels):
    return (preds == labels).mean()


def compute_f1(preds, labels):
    n_gold = n_pred = n_correct = 0
    for pred, label in zip(preds, labels):
        if pred != 0:
            n_pred += 1
        if label != 0:
            n_gold += 1
        if (pred != 0) and (label != 0) and (pred == label):
            n_correct += 1
    if n_correct == 0:
        return {'precision': 0.0, 'recall': 0.0, 'f1': 0.0}
    else:
        prec = n_correct * 1.0 / n_pred
        recall = n_correct * 1.0 / n_gold
        if prec + recall > 0:
            f1 = 2.0 * prec * recall / (prec + recall)
        else:
            f1 = 0.0
        return {'precision': prec, 'recall': recall, 'f1': f1}


def evaluate(model, device, eval_dataloader, eval_label_ids, num_labels, verbose=True):
    model.eval()
    eval_loss = 0
    nb_eval_steps = 0
    preds = []
    for input_ids, input_mask, segment_ids, label_ids in eval_dataloader:
        input_ids = input_ids.to(device)
        input_mask = input_mask.to(device)
        segment_ids = segment_ids.to(device)
        label_ids = label_ids.to(device)
        with torch.no_grad():
            logits = model(input_ids, segment_ids, input_mask, labels=None)
        loss_fct = CrossEntropyLoss()
        tmp_eval_loss = loss_fct(logits.view(-1, num_labels), label_ids.view(-1))
        eval_loss += tmp_eval_loss.mean().item()
        nb_eval_steps += 1
        if len(preds) == 0:
            preds.append(logits.detach().cpu().numpy())
        else:
            preds[0] = np.append(
                preds[0], logits.detach().cpu().numpy(), axis=0)

    eval_loss = eval_loss / nb_eval_steps
    preds = np.argmax(preds[0], axis=1)
    result = compute_f1(preds, eval_label_ids.numpy())
    result['accuracy'] = simple_accuracy(preds, eval_label_ids.numpy())
    result['eval_loss'] = eval_loss
    if verbose:
        logger.info("***** Eval results *****")
        for key in sorted(result.keys()):
            logger.info("  %s = %s", key, str(result[key]))
    return preds, result


def main(args):
    device = torch.device("cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu")
    n_gpu = torch.cuda.device_count()

    if args.gradient_accumulation_steps < 1:
        raise ValueError("Invalid gradient_accumulation_steps parameter: {}, should be >= 1".format(
                            args.gradient_accumulation_steps))
    args.train_batch_size = args.train_batch_size // args.gradient_accumulation_steps

    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if n_gpu > 0:
        torch.cuda.manual_seed_all(args.seed)

    if not args.do_train and not args.do_eval:
        raise ValueError("At least one of `do_train` or `do_eval` must be True.")

    if not os.path.exists(args.output_dir):
        os.makedirs(args.output_dir)
    if args.do_train:
        logger.addHandler(logging.FileHandler(os.path.join(args.output_dir, "train.log"), 'w'))
    else:
        logger.addHandler(logging.FileHandler(os.path.join(args.output_dir, "eval.log"), 'w'))
    logger.info(args)
    logger.info("device: {}, n_gpu: {}, 16-bits training: {}".format(
        device, n_gpu, args.fp16))


    processor = DataProcessor()
    label_list = processor.get_labels( args.fold, args.negative_label)
    label2id = {label: i for i, label in enumerate(label_list)}
    id2label = {i: label for i, label in enumerate(label_list)}
    num_labels = len(label_list)
    tokenizer = BertTokenizer.from_pretrained(args.model, do_lower_case=args.do_lower_case)

    special_tokens = {}
    if args.do_eval:
        eval_examples = processor.get_dev_examples(args.fold)
        eval_features = convert_examples_to_features(
            eval_examples, label2id, args.max_seq_length, tokenizer, special_tokens, args.feature_mode)
        logger.info("***** Dev *****")
        logger.info("  Num examples = %d", len(eval_examples))
        logger.info("  Batch size = %d", args.eval_batch_size)
        all_input_ids = torch.tensor([f.input_ids for f in eval_features], dtype=torch.long)
        all_input_mask = torch.tensor([f.input_mask for f in eval_features], dtype=torch.long)
        all_segment_ids = torch.tensor([f.segment_ids for f in eval_features], dtype=torch.long)
        all_label_ids = torch.tensor([f.label_id for f in eval_features], dtype=torch.long)
        eval_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
        eval_dataloader = DataLoader(eval_data, batch_size=args.eval_batch_size)
        eval_label_ids = all_label_ids

    if args.do_train:
        train_examples = processor.get_train_examples(args.fold)
        train_features = convert_examples_to_features(
                train_examples, label2id, args.max_seq_length, tokenizer, special_tokens, args.feature_mode)

        if args.train_mode == 'sorted' or args.train_mode == 'random_sorted':
            train_features = sorted(train_features, key=lambda f: np.sum(f.input_mask))
        else:
            random.shuffle(train_features)

        all_input_ids = torch.tensor([f.input_ids for f in train_features], dtype=torch.long)
        all_input_mask = torch.tensor([f.input_mask for f in train_features], dtype=torch.long)
        all_segment_ids = torch.tensor([f.segment_ids for f in train_features], dtype=torch.long)
        all_label_ids = torch.tensor([f.label_id for f in train_features], dtype=torch.long)
        train_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
        train_dataloader = DataLoader(train_data, batch_size=args.train_batch_size)
        train_batches = [batch for batch in train_dataloader]

        num_train_optimization_steps = \
            len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs

        logger.info("***** Training *****")
        logger.info("  Num examples = %d", len(train_examples))
        logger.info("  Batch size = %d", args.train_batch_size)
        logger.info("  Num steps = %d", num_train_optimization_steps)

        best_result = None
        eval_step = max(1, len(train_batches) // args.eval_per_epoch)
        lrs = [args.learning_rate] if args.learning_rate else \
            [1e-6, 2e-6, 3e-6, 5e-6, 1e-5, 2e-5, 3e-5, 5e-5]
        for lr in lrs:
            model = BertForSequenceClassification.from_pretrained(
                args.model, cache_dir=str(PYTORCH_PRETRAINED_BERT_CACHE), num_labels=num_labels)
            if args.fp16:
                model.half()
            model.to(device)
            if n_gpu > 1:
                model = torch.nn.DataParallel(model)

            param_optimizer = list(model.named_parameters())
            no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
            optimizer_grouped_parameters = [
                {'params': [p for n, p in param_optimizer
                            if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
                {'params': [p for n, p in param_optimizer
                            if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
            ]
            if args.fp16:
                try:
                    from apex.optimizers import FP16_Optimizer
                    from apex.optimizers import FusedAdam
                except ImportError:
                    raise ImportError("Please install apex from https://www.github.com/nvidia/apex"
                                      "to use distributed and fp16 training.")

                optimizer = FusedAdam(optimizer_grouped_parameters,
                                      lr=lr,
                                      bias_correction=False,
                                      max_grad_norm=1.0)
                if args.loss_scale == 0:
                    optimizer = FP16_Optimizer(optimizer, dynamic_loss_scale=True)
                else:
                    optimizer = FP16_Optimizer(optimizer, static_loss_scale=args.loss_scale)

            else:
                optimizer = BertAdam(optimizer_grouped_parameters,
                                     lr=lr,
                                     warmup=args.warmup_proportion,
                                     t_total=num_train_optimization_steps)

            start_time = time.time()
            global_step = 0
            tr_loss = 0
            nb_tr_examples = 0
            nb_tr_steps = 0
            for epoch in range(int(args.num_train_epochs)):
                model.train()
                logger.info("Start epoch #{} (lr = {})...".format(epoch, lr))
                if args.train_mode == 'random' or args.train_mode == 'random_sorted':
                    random.shuffle(train_batches)
                for step, batch in enumerate(train_batches):
                    batch = tuple(t.to(device) for t in batch)
                    input_ids, input_mask, segment_ids, label_ids = batch
                    loss = model(input_ids, segment_ids, input_mask, label_ids)
                    if n_gpu > 1:
                        loss = loss.mean()
                    if args.gradient_accumulation_steps > 1:
                        loss = loss / args.gradient_accumulation_steps

                    if args.fp16:
                        optimizer.backward(loss)
                    else:
                        loss.backward()

                    tr_loss += loss.item()
                    nb_tr_examples += input_ids.size(0)
                    nb_tr_steps += 1

                    if (step + 1) % args.gradient_accumulation_steps == 0:
                        if args.fp16:
                            lr_this_step = lr * \
                                warmup_linear(global_step/num_train_optimization_steps, args.warmup_proportion)
                            for param_group in optimizer.param_groups:
                                param_group['lr'] = lr_this_step
                        optimizer.step()
                        optimizer.zero_grad()
                        global_step += 1

                    if (step + 1) % eval_step == 0:
                        logger.info('Epoch: {}, Step: {} / {}, used_time = {:.2f}s, loss = {:.6f}'.format(
                                     epoch, step + 1, len(train_batches),
                                     time.time() - start_time, tr_loss / nb_tr_steps))
                        save_model = False
                        if args.do_eval:
                            preds, result = evaluate(model, device, eval_dataloader, eval_label_ids, num_labels)
                            model.train()
                            result['global_step'] = global_step
                            result['epoch'] = epoch
                            result['learning_rate'] = lr
                            result['batch_size'] = args.train_batch_size
                            logger.info("First 20 predictions:")
                            for pred, label in zip(preds[:20], eval_label_ids.numpy()[:20]):
                                sign = u'\u2713' if pred == label else u'\u2718'
                                logger.info("pred = %s, label = %s %s" % (id2label[pred], id2label[label], sign))
                            if (best_result is None) or (result[args.eval_metric] > best_result[args.eval_metric]):
                                best_result = result
                                save_model = True
                                logger.info("!!! Best dev %s (lr=%s, epoch=%d): %.2f" %
                                            (args.eval_metric, str(lr), epoch, result[args.eval_metric] * 100.0))
                        else:
                            save_model = True

                        if save_model:
                            model_to_save = model.module if hasattr(model, 'module') else model
                            output_model_file = os.path.join(args.output_dir, WEIGHTS_NAME)
                            output_config_file = os.path.join(args.output_dir, CONFIG_NAME)
                            torch.save(model_to_save.state_dict(), output_model_file)
                            model_to_save.config.to_json_file(output_config_file)
                            tokenizer.save_vocabulary(args.output_dir)
                            if best_result:
                                output_eval_file = os.path.join(args.output_dir, "eval_results.txt")
                                with open(output_eval_file, "w") as writer:
                                    for key in sorted(result.keys()):
                                        writer.write("%s = %s\n" % (key, str(result[key])))

    if args.do_eval:
        if args.eval_test:
            eval_examples = processor.get_test_examples(args.fold)
            eval_features = convert_examples_to_features(
                eval_examples, label2id, args.max_seq_length, tokenizer, special_tokens, args.feature_mode)
            logger.info("***** Test *****")
            logger.info("  Num examples = %d", len(eval_examples))
            logger.info("  Batch size = %d", args.eval_batch_size)
            all_input_ids = torch.tensor([f.input_ids for f in eval_features], dtype=torch.long)
            all_input_mask = torch.tensor([f.input_mask for f in eval_features], dtype=torch.long)
            all_segment_ids = torch.tensor([f.segment_ids for f in eval_features], dtype=torch.long)
            all_label_ids = torch.tensor([f.label_id for f in eval_features], dtype=torch.long)
            eval_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
            eval_dataloader = DataLoader(eval_data, batch_size=args.eval_batch_size)
            eval_label_ids = all_label_ids
        model = BertForSequenceClassification.from_pretrained(args.output_dir, num_labels=num_labels)
        if args.fp16:
            model.half()
        model.to(device)
        preds, result = evaluate(model, device, eval_dataloader, eval_label_ids, num_labels)
        with open(os.path.join(args.output_dir, "predictions.txt"), "w") as f:
            for ex, pred in zip(eval_examples, preds):
                f.write("%s\t%s\t%s\n" % (ex.guid, ex.label, id2label[pred]))
        with open(os.path.join(args.output_dir, "test_results.txt"), "w") as f:
            for key in sorted(result.keys()):
                f.write("%s = %s\n" % (key, str(result[key])))


In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument("--model", default=None, type=str, required=True)
#parser.add_argument("--data_dir", default=None, type=str, required=True,
#                    help="The input data dir. Should contain the .tsv files (or other data files) for the task.")
parser.add_argument("--output_dir", default=None, type=str, required=True,
                    help="The output directory where the model predictions and checkpoints will be written.")
parser.add_argument("--eval_per_epoch", default=10, type=int,
                    help="How many times it evaluates on dev set per epoch")
parser.add_argument("--max_seq_length", default=128, type=int,
                    help="The maximum total input sequence length after WordPiece tokenization. \n"
                          "Sequences longer than this will be truncated, and sequences shorter \n"
                          "than this will be padded.")
parser.add_argument("--negative_label", default="no_relation", type=str)
parser.add_argument("--do_train", action='store_true', help="Whether to run training.")
parser.add_argument("--train_mode", type=str, default='random_sorted', choices=['random', 'sorted', 'random_sorted'])
parser.add_argument("--do_eval", action='store_true', help="Whether to run eval on the dev set.")
parser.add_argument("--do_lower_case", action='store_true', help="Set this flag if you are using an uncased model.")
parser.add_argument("--eval_test", action="store_true", help="Whether to evaluate on final test set.")
parser.add_argument("--feature_mode", type=str, default="ner", choices=["text", "ner", "text_ner", "ner_text"])
parser.add_argument("--train_batch_size", default=32, type=int,
                    help="Total batch size for training.")
parser.add_argument("--eval_batch_size", default=8, type=int,
                    help="Total batch size for eval.")
parser.add_argument("--eval_metric", default="f1", type=str)
parser.add_argument("--learning_rate", default=None, type=float,
                    help="The initial learning rate for Adam.")
parser.add_argument("--num_train_epochs", default=3.0, type=float,
                    help="Total number of training epochs to perform.")
parser.add_argument("--warmup_proportion", default=0.1, type=float,
                    help="Proportion of training to perform linear learning rate warmup for. "
                          "E.g., 0.1 = 10%% of training.")
parser.add_argument("--no_cuda", action='store_true',
                    help="Whether not to use CUDA when available")
parser.add_argument('--seed', type=int, default=42,
                    help="random seed for initialization")
parser.add_argument('--gradient_accumulation_steps', type=int, default=1,
                    help="Number of updates steps to accumulate before performing a backward/update pass.")
parser.add_argument('--fp16', action='store_true',
                    help="Whether to use 16-bit float precision instead of 32-bit")
parser.add_argument('--loss_scale', type=float, default=0,
                    help="Loss scaling to improve fp16 numeric stability. Only used when fp16 set to True.\n"
                          "0 (default value): dynamic loss scaling.\n"
                          "Positive power of 2: static loss scaling value.\n")

parser.add_argument("--fold", default=0, type=int,
                    help="Data fold")


_StoreAction(option_strings=['--fold'], dest='fold', nargs=None, const=None, default=0, type=<class 'int'>, choices=None, help='Data fold', metavar=None)

## Фолд 1

In [ ]:
args = parser.parse_args(["--do_train", "--do_eval", "--eval_test", 
                          "--model", "/content/drive/MyDrive/Models/RUBERT_pt",  
                          "--output_dir", "/content/drive/MyDrive/Models/SpanBERT_Ru_14_02/fold0", 
                          "--fold", "0",
                          "--train_batch_size", "32",
                          "--eval_batch_size", "32",
                          "--learning_rate", "2e-5", 
                          "--num_train_epochs", "6",
                          "--max_seq_length", "128"])
print(args)

Namespace(do_eval=True, do_lower_case=False, do_train=True, eval_batch_size=32, eval_metric='f1', eval_per_epoch=10, eval_test=True, feature_mode='ner', fold=0, fp16=False, gradient_accumulation_steps=1, learning_rate=2e-05, loss_scale=0, max_seq_length=128, model='/content/drive/MyDrive/Models/RUBERT_pt', negative_label='no_relation', no_cuda=False, num_train_epochs=6.0, output_dir='/content/drive/MyDrive/Models/SpanBERT_Ru_14_02/fold0', seed=42, train_batch_size=32, train_mode='random_sorted', warmup_proportion=0.1)


In [ ]:
main(args)

02/21/2021 23:29:31 - INFO - __main__ - Namespace(do_eval=True, do_lower_case=False, do_train=True, eval_batch_size=32, eval_metric='f1', eval_per_epoch=10, eval_test=True, feature_mode='ner', fold=0, fp16=False, gradient_accumulation_steps=1, learning_rate=2e-05, loss_scale=0, max_seq_length=128, model='/content/drive/MyDrive/Models/RUBERT_pt', negative_label='no_relation', no_cuda=False, num_train_epochs=6.0, output_dir='/content/drive/MyDrive/Models/SpanBERT_Ru_14_02/fold0', seed=42, train_batch_size=32, train_mode='random_sorted', warmup_proportion=0.1)
02/21/2021 23:29:31 - INFO - __main__ - device: cuda, n_gpu: 1, 16-bits training: False
02/21/2021 23:29:31 - INFO - __main__ - 48 labels
02/21/2021 23:29:31 - INFO - __main__ - no_relation: 78.72%
02/21/2021 23:29:31 - INFO - __main__ - PARTICIPANT_IN: 3.21%
02/21/2021 23:29:31 - INFO - __main__ - WORKS_AS: 2.70%
02/21/2021 23:29:31 - INFO - __main__ - WORKPLACE: 2.09%
02/21/2021 23:29:31 - INFO - __main__ - TAKES_PLACE_IN: 1.39%
0

## Фолд 2

In [ ]:
args = parser.parse_args(["--do_train", "--do_eval", "--eval_test", 
                          "--model", "/content/drive/MyDrive/Models/RUBERT_pt",  
                          "--output_dir", "/content/drive/MyDrive/Models/SpanBERT_Ru_14_02/fold1", 
                          "--fold", "1",
                          "--train_batch_size", "32",
                          "--eval_batch_size", "32",
                          "--learning_rate", "2e-5", 
                          "--num_train_epochs", "6",
                          "--max_seq_length", "128"])
print(args)

Namespace(do_eval=True, do_lower_case=False, do_train=True, eval_batch_size=32, eval_metric='f1', eval_per_epoch=10, eval_test=True, feature_mode='ner', fold=1, fp16=False, gradient_accumulation_steps=1, learning_rate=2e-05, loss_scale=0, max_seq_length=128, model='/content/drive/MyDrive/Models/RUBERT_pt', negative_label='no_relation', no_cuda=False, num_train_epochs=6.0, output_dir='/content/drive/MyDrive/Models/SpanBERT_Ru_14_02/fold1', seed=42, train_batch_size=32, train_mode='random_sorted', warmup_proportion=0.1)


In [ ]:
main(args)

02/22/2021 00:18:44 - INFO - __main__ - Namespace(do_eval=True, do_lower_case=False, do_train=True, eval_batch_size=32, eval_metric='f1', eval_per_epoch=10, eval_test=True, feature_mode='ner', fold=1, fp16=False, gradient_accumulation_steps=1, learning_rate=2e-05, loss_scale=0, max_seq_length=128, model='/content/drive/MyDrive/Models/RUBERT_pt', negative_label='no_relation', no_cuda=False, num_train_epochs=6.0, output_dir='/content/drive/MyDrive/Models/SpanBERT_Ru_14_02/fold1', seed=42, train_batch_size=32, train_mode='random_sorted', warmup_proportion=0.1)
02/22/2021 00:18:44 - INFO - __main__ - device: cuda, n_gpu: 1, 16-bits training: False
02/22/2021 00:18:44 - INFO - __main__ - 48 labels
02/22/2021 00:18:44 - INFO - __main__ - no_relation: 78.77%
02/22/2021 00:18:44 - INFO - __main__ - PARTICIPANT_IN: 3.06%
02/22/2021 00:18:44 - INFO - __main__ - WORKS_AS: 2.77%
02/22/2021 00:18:44 - INFO - __main__ - WORKPLACE: 2.11%
02/22/2021 00:18:44 - INFO - __main__ - ORIGINS_FROM: 1.44%
02/

## Фолд 4

In [ ]:
args = parser.parse_args(["--do_train", "--do_eval", "--eval_test", 
                          "--model", "/content/drive/MyDrive/Models/RUBERT_pt",  
                          "--output_dir", "/content/drive/MyDrive/Models/SpanBERT_Ru_14_02/fold3", 
                          "--fold", "3",
                          "--train_batch_size", "32",
                          "--eval_batch_size", "32",
                          "--learning_rate", "2e-5", 
                          "--num_train_epochs", "6",
                          "--max_seq_length", "128"])
print(args)

Namespace(do_eval=True, do_lower_case=False, do_train=True, eval_batch_size=32, eval_metric='f1', eval_per_epoch=10, eval_test=True, feature_mode='ner', fold=3, fp16=False, gradient_accumulation_steps=1, learning_rate=2e-05, loss_scale=0, max_seq_length=128, model='/content/drive/MyDrive/Models/RUBERT_pt', negative_label='no_relation', no_cuda=False, num_train_epochs=6.0, output_dir='/content/drive/MyDrive/Models/SpanBERT_Ru_14_02/fold3', seed=42, train_batch_size=32, train_mode='random_sorted', warmup_proportion=0.1)


In [ ]:
main(args)

02/22/2021 01:56:40 - INFO - __main__ - Namespace(do_eval=True, do_lower_case=False, do_train=True, eval_batch_size=32, eval_metric='f1', eval_per_epoch=10, eval_test=True, feature_mode='ner', fold=3, fp16=False, gradient_accumulation_steps=1, learning_rate=2e-05, loss_scale=0, max_seq_length=128, model='/content/drive/MyDrive/Models/RUBERT_pt', negative_label='no_relation', no_cuda=False, num_train_epochs=6.0, output_dir='/content/drive/MyDrive/Models/SpanBERT_Ru_14_02/fold3', seed=42, train_batch_size=32, train_mode='random_sorted', warmup_proportion=0.1)
02/22/2021 01:56:40 - INFO - __main__ - device: cuda, n_gpu: 1, 16-bits training: False
02/22/2021 01:56:40 - INFO - __main__ - 48 labels
02/22/2021 01:56:40 - INFO - __main__ - no_relation: 78.83%
02/22/2021 01:56:40 - INFO - __main__ - PARTICIPANT_IN: 3.20%
02/22/2021 01:56:40 - INFO - __main__ - WORKS_AS: 2.80%
02/22/2021 01:56:40 - INFO - __main__ - WORKPLACE: 2.21%
02/22/2021 01:56:40 - INFO - __main__ - POINT_IN_TIME: 1.40%
02

# На официальном разбиении создателей датасета

In [ ]:
args = parser.parse_args(["--do_train", "--do_eval", "--eval_test", 
                          "--model", "/content/drive/MyDrive/My_models/BERT_MULTILING_cased",  
                          "--output_dir", "/content/drive/MyDrive/My_models/SpanBERT_mult_final", 
                          "--fold", "0",
                          "--train_batch_size", "32",
                          "--eval_batch_size", "32",
                          "--learning_rate", "2e-5", 
                          "--num_train_epochs", "7",
                          "--max_seq_length", "128"])
print(args)

Namespace(do_eval=True, do_lower_case=False, do_train=True, eval_batch_size=32, eval_metric='f1', eval_per_epoch=10, eval_test=True, feature_mode='ner', fold=0, fp16=False, gradient_accumulation_steps=1, learning_rate=2e-05, loss_scale=0, max_seq_length=128, model='/content/drive/MyDrive/My_models/BERT_MULTILING_cased', negative_label='no_relation', no_cuda=False, num_train_epochs=7.0, output_dir='/content/drive/MyDrive/My_models/SpanBERT_mult_final', seed=42, train_batch_size=32, train_mode='random_sorted', warmup_proportion=0.1)


In [ ]:
main(args)

04/26/2021 11:11:12 - INFO - __main__ - Namespace(do_eval=True, do_lower_case=False, do_train=True, eval_batch_size=32, eval_metric='f1', eval_per_epoch=10, eval_test=True, feature_mode='ner', fold=0, fp16=False, gradient_accumulation_steps=1, learning_rate=2e-05, loss_scale=0, max_seq_length=128, model='/content/drive/MyDrive/My_models/BERT_MULTILING_cased', negative_label='no_relation', no_cuda=False, num_train_epochs=7.0, output_dir='/content/drive/MyDrive/My_models/SpanBERT_mult_final', seed=42, train_batch_size=32, train_mode='random_sorted', warmup_proportion=0.1)
04/26/2021 11:11:12 - INFO - __main__ - device: cuda, n_gpu: 1, 16-bits training: False
04/26/2021 11:11:12 - INFO - __main__ - 50 labels
04/26/2021 11:11:12 - INFO - __main__ - no_relation: 77.07%
04/26/2021 11:11:12 - INFO - __main__ - WORKS_AS: 3.87%
04/26/2021 11:11:12 - INFO - __main__ - PARTICIPANT_IN: 3.42%
04/26/2021 11:11:12 - INFO - __main__ - POINT_IN_TIME: 1.78%
04/26/2021 11:11:12 - INFO - __main__ - TAKES_

Второе обучение

In [ ]:
args = parser.parse_args(["--do_train", "--do_eval", "--eval_test", 
                          "--model", "/content/drive/MyDrive/Models/RUBERT_pt",  
                          "--output_dir", "/content/drive/MyDrive/My_models/SpanBERT_ru_NREL/attempt_2", 
                          "--fold", "0",
                          "--train_batch_size", "32",
                          "--eval_batch_size", "32",
                          "--learning_rate", "1e-5", 
                          "--num_train_epochs", "6",
                          "--max_seq_length", "128"])
print(args)

Namespace(do_eval=True, do_lower_case=False, do_train=True, eval_batch_size=32, eval_metric='f1', eval_per_epoch=10, eval_test=True, feature_mode='ner', fold=0, fp16=False, gradient_accumulation_steps=1, learning_rate=1e-05, loss_scale=0, max_seq_length=128, model='/content/drive/MyDrive/Models/RUBERT_pt', negative_label='no_relation', no_cuda=False, num_train_epochs=6.0, output_dir='/content/drive/MyDrive/My_models/SpanBERT_ru_NREL/attempt_2', seed=42, train_batch_size=32, train_mode='random_sorted', warmup_proportion=0.1)


In [ ]:
main(args)

04/24/2021 12:59:11 - INFO - __main__ - Namespace(do_eval=True, do_lower_case=False, do_train=True, eval_batch_size=32, eval_metric='f1', eval_per_epoch=10, eval_test=True, feature_mode='ner', fold=0, fp16=False, gradient_accumulation_steps=1, learning_rate=1e-05, loss_scale=0, max_seq_length=128, model='/content/drive/MyDrive/Models/RUBERT_pt', negative_label='no_relation', no_cuda=False, num_train_epochs=6.0, output_dir='/content/drive/MyDrive/My_models/SpanBERT_ru_NREL/attempt_2', seed=42, train_batch_size=32, train_mode='random_sorted', warmup_proportion=0.1)
04/24/2021 12:59:11 - INFO - __main__ - device: cuda, n_gpu: 1, 16-bits training: False
04/24/2021 12:59:11 - INFO - __main__ - 50 labels
04/24/2021 12:59:11 - INFO - __main__ - no_relation: 77.07%
04/24/2021 12:59:11 - INFO - __main__ - WORKS_AS: 3.87%
04/24/2021 12:59:11 - INFO - __main__ - PARTICIPANT_IN: 3.42%
04/24/2021 12:59:11 - INFO - __main__ - POINT_IN_TIME: 1.78%
04/24/2021 12:59:11 - INFO - __main__ - TAKES_PLACE_I

AttributeError: ignored

# Анализ результатов


In [ ]:
import sklearn
import json

In [ ]:
#fold0
path_pred0 = "/content/drive/MyDrive/Models/SpanBERT_Ru_14_02/official_split/predictions.txt"
with open(path_pred0, 'r') as f:
  pred0 = f.readlines()

id_test0 = [line.strip().split('\t')[0] for line in pred0] 
y_true0 = [line.strip().split('\t')[1] for line in pred0]
y_pred0 = [line.strip().split('\t')[2] for line in pred0]

print("FOLD 0:")
print(sklearn.metrics.classification_report(y_true0, y_pred0))

FileNotFoundError: ignored

In [ ]:
# проверим, учитывается ли negative label при подсчете метрики

sklearn.metrics.f1_score(y_true0, y_pred0, labels = list(set(y_true0) - set(['no_relation'])), average = 'micro')

0.7703464947622884

In [ ]:
# в тесте при обучении также 0.77 => все ок
# c 'no_relation' f1 = 0.91

In [ ]:
list(set(y_true0) - set(['no_relation']))

['PRICE_OF',
 'OWNERSHIP',
 'START_TIME',
 'AWARDED_WITH',
 'SUBEVENT_OF',
 'SPOUSE',
 'DATE_DEFUNCT_IN',
 'PARENT_OF',
 'FOUNDED_BY',
 'KNOWS',
 'WORKPLACE',
 'PLACE_OF_BIRTH',
 'WORKS_AS',
 'DATE_FOUNDED_IN',
 'HAS_CAUSE',
 'DATE_OF_CREATION',
 'ALTERNATIVE_NAME',
 'ORGANIZES',
 'RELIGION_OF',
 'LOCATED_IN',
 'POINT_IN_TIME',
 'AGE_DIED_AT',
 'END_TIME',
 'PART_OF',
 'ORIGINS_FROM',
 'IDEOLOGY_OF',
 'DATE_OF_DEATH',
 'PENALIZED_AS',
 'MEMBER_OF',
 'PRODUCES',
 'TAKES_PLACE_IN',
 'OWNER_OF',
 'MEDICAL_CONDITION',
 'PARTICIPANT_IN',
 'SIBLING',
 'PLACE_RESIDES_IN',
 'SCHOOLS_ATTENDED',
 'EXPENDITURE',
 'DATE_OF_BIRTH',
 'PLACE_OF_DEATH',
 'ABBREVIATION',
 'RELATIVE',
 'SUBORDINATE_OF',
 'CONVICTED_OF',
 'AGE_IS',
 'CAUSE_OF_DEATH',
 'HEADQUARTERED_IN']

In [ ]:
#fold1
path_pred1 = '/content/drive/MyDrive/Models/SpanBERT_Ru_14_02/fold1/predictions.txt'
with open(path_pred1, 'r') as f:
  pred1 = f.readlines()

id_test1 = [line.strip().split('\t')[0] for line in pred1] 
y_true1 = [line.strip().split('\t')[1] for line in pred1]
y_pred1 = [line.strip().split('\t')[2] for line in pred1]

print("FOLD 2:")
print(sklearn.metrics.classification_report(y_true1, y_pred1))

FOLD 2:
                   precision    recall  f1-score   support

     ABBREVIATION       0.50      0.57      0.53         7
      AGE_DIED_AT       0.86      0.75      0.80         8
           AGE_IS       0.84      0.90      0.87        52
 ALTERNATIVE_NAME       0.71      0.52      0.60        33
     AWARDED_WITH       0.77      0.77      0.77        13
   CAUSE_OF_DEATH       1.00      0.43      0.60         7
     CONVICTED_OF       0.82      0.75      0.78        12
  DATE_DEFUNCT_IN       0.00      0.00      0.00         2
  DATE_FOUNDED_IN       0.00      0.00      0.00         4
    DATE_OF_BIRTH       1.00      1.00      1.00        13
 DATE_OF_CREATION       0.77      0.77      0.77        13
    DATE_OF_DEATH       1.00      0.83      0.91        12
         END_TIME       0.00      0.00      0.00         2
      EXPENDITURE       0.00      0.00      0.00         1
       FOUNDED_BY       0.73      0.67      0.70        12
        HAS_CAUSE       0.17      0.14      0.1

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#fold2
path_pred2 = '/content/drive/MyDrive/Models/SpanBERT_Ru_14_02/fold2/predictions.txt'
with open(path_pred2, 'r') as f:
  pred2 = f.readlines()

id_test2 = [line.strip().split('\t')[0] for line in pred2] 
y_true2 = [line.strip().split('\t')[1] for line in pred2]
y_pred2 = [line.strip().split('\t')[2] for line in pred2]

print("FOLD 3:")
print(sklearn.metrics.classification_report(y_true2, y_pred2))

FOLD 2:
                   precision    recall  f1-score   support

     ABBREVIATION       0.67      0.33      0.44         6
      AGE_DIED_AT       0.80      0.50      0.62         8
           AGE_IS       0.85      0.92      0.88        48
 ALTERNATIVE_NAME       0.78      0.49      0.60        37
     AWARDED_WITH       0.87      0.87      0.87        23
   CAUSE_OF_DEATH       0.00      0.00      0.00         3
     CONVICTED_OF       0.82      0.90      0.86        10
  DATE_FOUNDED_IN       0.67      0.50      0.57         4
    DATE_OF_BIRTH       0.89      1.00      0.94         8
 DATE_OF_CREATION       0.82      0.56      0.67        16
    DATE_OF_DEATH       0.80      0.89      0.84         9
      EXPENDITURE       0.00      0.00      0.00         2
       FOUNDED_BY       1.00      0.50      0.67         6
        HAS_CAUSE       0.64      0.60      0.62        15
 HEADQUARTERED_IN       0.67      0.84      0.75        37
      IDEOLOGY_OF       0.81      0.81      0.8

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#fold3
path_pred3 = '/content/drive/MyDrive/Models/SpanBERT_Ru_14_02/fold3/predictions.txt'
with open(path_pred3, 'r') as f:
  pred3 = f.readlines()

id_test3 = [line.strip().split('\t')[0] for line in pred3] 
y_true3 = [line.strip().split('\t')[1] for line in pred3]
y_pred3 = [line.strip().split('\t')[2] for line in pred3]

print("FOLD 4:")
print(sklearn.metrics.classification_report(y_true3, y_pred3))

FOLD 4:
                   precision    recall  f1-score   support

     ABBREVIATION       0.83      0.38      0.53        13
      AGE_DIED_AT       0.86      0.60      0.71        10
           AGE_IS       0.89      0.92      0.90        59
 ALTERNATIVE_NAME       0.71      0.48      0.57        42
     AWARDED_WITH       0.60      0.69      0.64        13
   CAUSE_OF_DEATH       0.50      0.25      0.33         4
     CONVICTED_OF       0.75      0.60      0.67        10
  DATE_DEFUNCT_IN       0.00      0.00      0.00         2
  DATE_FOUNDED_IN       0.00      0.00      0.00         2
    DATE_OF_BIRTH       1.00      0.80      0.89        15
 DATE_OF_CREATION       0.86      0.92      0.89        13
    DATE_OF_DEATH       1.00      0.75      0.86         8
      EXPENDITURE       0.00      0.00      0.00         1
       FOUNDED_BY       0.46      0.50      0.48        12
        HAS_CAUSE       0.40      0.60      0.48        10
 HEADQUARTERED_IN       0.72      0.93      0.8

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
